### Text file, here english wikipedia

In [9]:
import torch

In [10]:
import torch.utils.data as data

In [11]:
from ipywidgets import FloatProgress
from IPython.display import display

In [12]:
wiki_file = "/data/datasets/enwiki_pre.txt"

In [13]:
def countlines(file):
    c = 0
    for line in file:
        c += 1
    file.seek(0)
    return c

In [15]:
def createDictionnary(text_file, min_count=30):
    word2id = {}
    wordCount = {}
    i = 0
    with open(text_file,encoding="ISO-8859-1") as f:
        c = countlines(f)
        prog = FloatProgress(min=0, max=c)
        display(prog)
        for j, line in enumerate(f):
            if j % 1000 == 0:
                prog.value += 1000
            #update_progress(j / c)
            #print("%2.2f" %(j/c*100), end='')            
            for word in line.split(' '):
                if not word in word2id:
                    word2id[word] = i
                    i += 1
                
                if not word in wordCount:
                    wordCount[word] = 1
                else:
                    wordCount[word] += 1
    word2id = {w:word2id[w] for w in word if wordCount[w] >= min_count}
    return word2id

In [18]:
class contextDataset(data.Dataset):
    def __init__(self, text_file, word2id=None):
        if word2id is None:
            self.word2id, _ = createDictionnary(text_file)
        self.text = []
        with open(text_file, encoding="ISO-8859-1") as f:
            print("Reading text file")
            prog = FloatProgress(min=0, max=countlines(f))
            for line in f:
                prog.value += 1
                self.text.extend([w for w in line.split(' ') if w in word2id])
            
        self.word2id = word2id
        
    def __len__(self):
        return len(text)
    
    def __getitem__(self, i):
        target = torch.LongTensor([self.word2id[self.text[i]]])
        if i >= 2 and i < len(text) - 2:
            context = [self.text[i - 2], self.text[i - 1],
                      self.text[i + 1], self.text[i + 2]]
        elif i == 1:
            context = [self.text[i - 1], self.text[i + 1],
                      self.text[i + 2], self.text[i + 3]]
        elif i == 0:
            context = [self.text[i + 1], self.text[i + 2], 
                       self.text[i + 3], self.text[i+4]]
        elif i >= len(text) - 2:
            context = [self.text[i-4], self.text[i-3] ,self.text[i - 2], self.text[i - 1]]
        else:
            print("ERROOR :", i, '/', len(self.text))
            return None
        context = [self.word2id[c] for c in context]
        
        neg = random.choices(list(word2id.values()), k=1)
        
        return target, context, neg

In [ ]:
cd = contextDataset(wiki_file)

FloatProgress(value=0.0, max=85778920.0)

In [5]:
a = [1,2,3]
b = [4,5,6]

In [2]:
%%timeit
a.extend(b)

84.4 ns ± 1.72 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [8]:
%%timeit
c = a + b

60.1 ns ± 0.987 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
